# Лабораторная работа 9
# Создание текстового файла большого объема с помощью генераторной функции
## Домаш Иван , 06.06.2025


## Функция randint из модуля random


In [3]:
from random import randint

In [4]:
?randint


Signature: randint(a, b)
Docstring:
Return random integer in range [a, b], including both end points.
        
File:      d:\python\envs\widgets_env\lib\random.py
Type:      method

In [5]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
f'Код символа: {(c:=randint(*latin_code))}, символ: {chr(c)}'


'Код символа: 118, символ: v'

## Функция process_time из модуля time


In [6]:
from time import process_time
?process_time


Docstring:
process_time() -> float

Process time for profiling: sum of the kernel and user-space CPU time.
Type:      builtin_function_or_method

In [7]:
start = process_time()
[x**2 for x in range(1_000_000)]
end = process_time()
print(f'{end - start} секунд')

0.140625 секунд


## Задание 9.1. Запись данных в текстовый файл



In [8]:
file_name = 'test.txt'
f = open(file_name, 'w')

In [9]:
lines_list = [f'{x}\n' for x in 'test']
lines_list


['t\n', 'e\n', 's\n', 't\n']

In [10]:
f.writelines(lines_list)

In [11]:
f.close()

In [12]:
with open(file_name,'r') as f:
 for line in f:
  print(line, end='')


t
e
s
t


In [13]:
lines_gen_expr = (f'{x}\n' for x in range(10**6))
def lines_gen_fun(number_lines=10**6):
 yield from (f'{x}\n' for x in range(number_lines))
with open(file_name,'w') as f:
 f.writelines(lines_gen_expr)
 f.writelines(lines_gen_fun())


In [14]:
def lines_gen_fun(number_lines = 10**6):
  file_size = 0
  for x in range(number_lines):
   line = f'{x}\n'
   yield line
   file_size += len(line)
  print(f'{file_size/1024**2} Mb')

with open(file_name,'w') as f:
 f.writelines(lines_gen_fun())


6.569757461547852 Mb


In [15]:
len('f'), len('f'.encode('utf8')), len('ц'), len('ц'.encode('utf8'))


(1, 1, 1, 2)

In [16]:
with open(file_name,'w') as f:
 start = process_time()
 f.writelines(lines_gen_fun())
 end = process_time()
 print(end - start, 'секунд')

6.569757461547852 Mb
0.90625 секунд


In [17]:
#   number_lines - количество генерируемых строк (по умолчанию 1 миллион)
#   status - флаг отображения прогресса генерации (по умолчанию False)
def lines_gen_fun(number_lines=10**6, status=False):
 file_size = 0 # Формирование строки: число + символ переноса строки
 for x in range(number_lines): 
  line = f'{x}\n'# Формирование строки: число + символ переноса строки
  yield line # Возврат сгенерированной строки (функция-генератор)
  file_size += len(line)  # Подсчет общего размера сгенерированных данных (в байтах)
 # отображение статуса записи в файл в процентах
  if status:
   status_number = x/number_lines*100
 # \r возврат курсора в начало строки для перезаписи
 print(f'\r{int(status_number)}%', end='', flush=True)
 print(f'\n {file_size/1024**2} Mb') 


In [18]:
with open(file_name,'w') as f:
 start = process_time()
 f.writelines(lines_gen_fun(3*10**3, status=True))
 end = process_time()
 print(f'{end - start} секунд')


99%
 0.013246536254882812 Mb
0.0 секунд


## Задание 9.2. Генерация строк

In [19]:
latin_code = (ord('a'), ord('z')) 
number_words = (10, 20)
number_symbols = (5, 10)


In [20]:
len_word = randint(*number_symbols)
word = ''.join([chr(randint(*latin_code)) for i in range(len_word)])
word

'wxmxmigulo'

In [36]:
import random
from string import ascii_uppercase, digits

def generate_random_word():
    alphabets = {
        'latin': ascii_uppercase,  
        'digits': digits,           
        'cyrillic': 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'  
    }
    
    alphabet_name = random.choice(list(alphabets.keys()))
    alphabet = alphabets[alphabet_name]
    word_length = random.randint(3, 13)
    word = ''.join(random.choice(alphabet) for _ in range(word_length))
    
    return word

def generate_random_sentence():
    num_words = random.randint(2, 7)
    sentence = ' '.join(generate_random_word() for _ in range(num_words))
    return sentence

for i in range(5):  
    print(generate_random_sentence())

ТНОИЦЮС ГТХАКЖСМ ЙЗБШДЮСЪЧЖНХ 990331318856 ИШОШПЛШ
87333 50844 15133743
WXGNYDNCPLFJ БЭЩТЖЫВЦ ЮЮЯЪЕЪЦПДРДЭ ХМШЩЗШБТПСЛМЦ ДУСМИТА ЧЧКЕЪЬП
7581 QMZD 516 УЧЪ 288 ЬЦПЙЧЗАХЯЮ WOIEX
97204760 SYGIKKNYAE 60236491851 ЕЛИТНЦРЦСХЯН 34902069889 FQCE


In [43]:
def lines_gen_fun_v2(file_size, type_symbols='mixed',min_words=10,max_words = 20,min_symbols = 5,  max_symbols=10,  status=False):

    alphabets = {
        'latin': ascii_uppercase,
        'digits': digits,
        'cyrillic': 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ',
        'mixed': ascii_uppercase + digits + 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
    }
    
    current_size = 0  
    target_bytes = file_size * 1024**2  
    line_count = 0
    
    while current_size < target_bytes:
        
        words = []
        for _ in range(random.randint(min_words, max_words)):
            word_length = random.randint(min_symbols, max_symbols)
            word = ''.join(random.choice(alphabets[type_symbols]) for _ in range(word_length))
            words.append(word)
        
        line = ' '.join(words) + '\n'
        line_size = len(line.encode('utf-8'))  #  кириллицы
        
        yield line
        current_size += line_size
    
    if status:
        print(f'\nend {current_size/1024**2:.2f} Мб')


In [46]:

gen = lines_gen_fun_v2(file_size=1, type_symbols='mixed', status=True)
with open('random_text.txt', 'w', encoding='utf-8') as f:
    for line in gen:
        f.write(line)


end 1.00 Мб


In [47]:
gen = lines_gen_fun_v2(file_size=0.5, type_symbols='latin', status=True)
with open('test_latin.txt', 'w', encoding='utf-8') as f:
    for line in gen:
        f.write(line)


end 0.50 Мб


In [48]:
gen = lines_gen_fun_v2(file_size=1, type_symbols='digits', status=True)
with open('test_digits.txt', 'w', encoding='utf-8') as f:
    for line in gen:
        f.write(line)


end 1.00 Мб


## 9.3 Запись сгенерированных строк в текстовый файл


In [52]:
def file_gen(
    file_name: str,
    file_size: int,
    type_symbols: str = 'latin',
    number_words: tuple[int, int] = (10, 20),
    number_symbols: tuple[int, int] = (5, 10),
    status: bool = True
):
    start_time = process_time()
    generator = lines_gen_fun_v2(
        file_size=file_size,
        type_symbols=type_symbols,
        min_words=number_words[0],  
        max_words=number_words[1],  
        min_symbols=number_symbols[0],  
        max_symbols=number_symbols[1],  
        status=status
    )
    with open(file_name, 'w', encoding='utf-8') as f:
        for line in generator:
            f.write(line)
    end_time = process_time()
    elapsed_time = end_time - start_time
    import os
    actual_size = os.path.getsize(file_name) / (1024 ** 2)
    
    print(f" {file_name}")
    print(f"Фактический размер: {actual_size:.2f} Мб")
    print(f"Время выполнения: {elapsed_time:.2f} секунд")

In [53]:
file_gen(
        file_name="file_gen.txt",
        file_size=1,  
        type_symbols="mixed",
        number_words=(10, 20),
        number_symbols=(5, 10)
    )


end 1.00 Мб
 file_gen.txt
Фактический размер: 1.01 Мб
Время выполнения: 0.45 секунд


In [54]:
file_gen(
    file_name="test_cyrillic.txt",
    file_size=0.1,  # 10 MB
    type_symbols="cyrillic",
    number_words=(1, 5),
    number_symbols=(5, 12)
)


end 0.10 Мб
 test_cyrillic.txt
Фактический размер: 0.10 Мб
Время выполнения: 0.03 секунд
